# K Nearest Neighbors on ROS IMU Data

Here we look at applying K-nearest neighbor classification on a dataset of IMU and odometry data collected on the TurtleBot3 Waffle Pi using ROS. 

In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier

## Data preparation and preprocessing

In [3]:
folder = "~/Desktop/Design Teams/QMIND"
file = "LargerSet_4T.csv"

df = pd.read_csv(os.path.join(folder, file))

df = df.rename(columns={'Unnamed: 0': 'Seq'})
df = df.set_index('Seq')
df.head(10)

In [5]:
# check & remove null values
print(df.isnull().sum().sum())
df.shape

0


(153052, 121)

In [6]:
# view features 
for col in df.columns:
    print(col)

OdomPosX
OdomPosY
OdomOrientZ
OdomOrientW
OdomLinX
OdomAngZ
ImuOrientX
ImuOrientY
ImuOrientZ
ImuOrientW
ImuAngVelX
ImuAngVelY
ImuAngVelZ
ImuAccelX
ImuAccelY
ImuAccelZ
TimeDelta2
TimeDelta4
TimeDelta8
TimeDelta16
TimeDelta32
TimeDelta64
OdomPosXDelta2
OdomPosXDelta4
OdomPosXDelta8
OdomPosXDelta16
OdomPosXDelta32
OdomPosXDelta64
OdomPosYDelta2
OdomPosYDelta4
OdomPosYDelta8
OdomPosYDelta16
OdomPosYDelta32
OdomPosYDelta64
OdomOrientZDelta2
OdomOrientZDelta4
OdomOrientZDelta8
OdomOrientZDelta16
OdomOrientZDelta32
OdomOrientZDelta64
OdomOrientWDelta2
OdomOrientWDelta4
OdomOrientWDelta8
OdomOrientWDelta16
OdomOrientWDelta32
OdomOrientWDelta64
OdomLinXDelta2
OdomLinXDelta4
OdomLinXDelta8
OdomLinXDelta16
OdomLinXDelta32
OdomLinXDelta64
OdomAngZDelta2
OdomAngZDelta4
OdomAngZDelta8
OdomAngZDelta16
OdomAngZDelta32
OdomAngZDelta64
ImuOrientXDelta2
ImuOrientXDelta4
ImuOrientXDelta8
ImuOrientXDelta16
ImuOrientXDelta32
ImuOrientXDelta64
ImuOrientYDelta2
ImuOrientYDelta4
ImuOrientYDelta8
ImuOrientYDelt

### Split to training and testing

In [11]:
trainColumns = df.query('Test != 3 and Test != 4', inplace = False)
testColumns = df.query('Test == 3 or Test == 4', inplace = False)

trainColumns.head()

,OdomPosX,OdomPosY,OdomOrientZ,OdomOrientW,OdomLinX,OdomAngZ,ImuOrientX,ImuOrientY,ImuOrientZ,ImuOrientW,...,ImuAccelYDelta64,ImuAccelZDelta2,ImuAccelZDelta4,ImuAccelZDelta8,ImuAccelZDelta16,ImuAccelZDelta32,ImuAccelZDelta64,Speed,Terrain,Test
Seq,,,,,,,,,,,,,,,,,,,,,
0,-4.032375,18.373403,0.093063,0.995660,0.152541,0.000100,-0.011154,0.006497,-0.101977,-0.994699,...,-0.325611,0.854132,1.829170,1.553837,0.088286,0.581791,0.795174,15,MitTile,8
1,-4.031664,18.373537,0.093066,0.995660,0.153398,0.000885,-0.010974,0.006144,-0.101975,-0.994703,...,-0.550966,-1.128866,1.206079,-0.420781,-1.149217,1.366490,1.614889,15,MitTile,8
2,-4.029840,18.373881,0.093065,0.995660,0.151111,0.000587,-0.010794,0.005792,-0.101973,-0.994707,...,-0.455497,-1.246181,-0.392050,-0.610222,-1.326387,-0.668281,-0.390254,15,MitTile,8
3,-4.028017,18.374225,0.093065,0.995660,0.148823,0.000288,-0.010391,0.005561,-0.101971,-0.994713,...,-0.360327,-0.936731,-2.065598,-0.490212,-1.177648,-2.393602,-0.495001,15,MitTile,8
4,-4.026193,18.374569,0.093065,0.995660,0.146536,-0.000010,-0.010383,0.007426,-0.101984,-0.994698,...,-0.005387,-0.942418,-2.188600,-0.359429,-1.344045,-1.821090,-0.914885,15,MitTile,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153047,-6.710423,13.805720,0.151013,0.988532,0.145356,-0.003765,-0.013807,0.009487,-0.159757,-0.987010,...,-0.058957,-0.462679,-1.469441,-2.051232,-0.265158,-0.201711,-1.783680,15,MitTile,6
153048,-6.710423,13.805720,0.151013,0.988532,0.145356,-0.003765,-0.013613,0.009750,-0.159733,-0.987014,...,-0.350152,-0.718261,-1.828571,-1.830667,-2.195483,-0.560842,-0.710480,15,MitTile,6
153049,-6.710423,13.805720,0.151013,0.988532,0.145356,-0.003765,-0.012885,0.009198,-0.159698,-0.987034,...,-0.726640,1.095946,0.633266,0.204105,-0.080804,0.979229,1.491588,15,MitTile,6


### Drop label columns and apply the label encoder

In [7]:
le = preprocessing.LabelEncoder()

In [12]:
X_train = trainColumns.drop(columns=['Speed', 'Terrain'])
columnsToDrop = [x for x in X_train.columns.tolist() if 'Time' in x or 'Delta' not in x or 'Orient' in x or 'Pos' in x]
X_train = X_train.drop(columns=columnsToDrop)
y_train = le.fit_transform(trainColumns['Terrain'])

X_test = testColumns.drop(columns=['Speed', 'Terrain'])
columnsToDrop = [x for x in X_test.columns.tolist() if 'Time' in x or 'Delta' not in x or 'Orient' in x or 'Pos' in x]
X_test = X_test.drop(columns=columnsToDrop)
y_test = le.fit_transform(testColumns['Terrain'])

In [14]:
# Scale features for improved performance

scaler = StandardScaler()

scaledFeatures = scaler.fit_transform(X_train.values)

X_train = pd.DataFrame(scaledFeatures, columns = X_train.columns.tolist())

scaledFeatures = scaler.fit_transform(X_test.values)

X_test = pd.DataFrame(scaledFeatures, columns =X_test.columns.tolist())

## Model definition and application

In [16]:
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [17]:
y_pred = model.predict(X_test)

### View results with confusion matrix

In [18]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 7785  2026   835    72]
 [ 1356  1826   264  1995]
 [ 5822  1530  2456   100]
 [  132   291     5 11387]]
              precision    recall  f1-score   support

           0       0.52      0.73      0.60     10718
           1       0.32      0.34      0.33      5441
           2       0.69      0.25      0.36      9908
           3       0.84      0.96      0.90     11815

    accuracy                           0.62     37882
   macro avg       0.59      0.57      0.55     37882
weighted avg       0.63      0.62      0.59     37882

